In [1]:
from copy import copy

test = ['if','sth',['add',2,3]]
example = [["IF", ["ISZERO", ["EQ", ["STORAGE", 160, 0, 0], "CALLER"]], [["REVERT", 0]], [["IF", ["ISZERO", ["EXTCODESIZE", ["MASK_SHL", 160, 0, 0, ["cd", 4]]]], [["REVERT", 0]], [["CALL", ["ADD", -710, "GAS"], ["MASK_SHL", 160, 0, 0, ["cd", 4]], 0, 1981353871, 'null'], ["IF", ["ISZERO", "ext_call.success"], [["REVERT", 0]], [["IF", ["ISZERO", "ext_call.return_data"], [["REVERT", 0]], [["STORE", 160, 0, 12, [], ["MASK_SHL", 160, 0, 0, ["cd", 4]]], ["STOP"]]]]]]]]]]
terminal = []

class Node:

    def __repr__(self):
        if len(self.children) == 0:
            return self.name
        else:
            return f'( {self.name} '+ (' '.join([str(c) for c in self.children]))+' )'

    def __init__(self, exp, parent=None):
        self.parent = parent
        self.children = []

        if type(exp) == list and len(exp)>0:
            self.name = exp[0] if type(exp[0]) == str else 'list'
            if self.name != 'list':
                for e in exp[1:]:
                    self.children.append(Node(e, parent=self))
            else:
                for e in exp:
                    self.children.append(Node(e, parent=self))

        else:
            self.name=str(exp)
            global terminal
            terminal.append(self)

    def downward_paths(self, max_len= None ):
        result = []
        vocab = set()

        def dfs(node, path_so_far, ignore):

            path_so_far = copy(path_so_far)
            path_so_far.append(node.name)
                
            if len(node.children) == 0:
                result = [path_so_far]
            else:
                result = []
            
            for c in node.children:
                if c is not ignore:
                    result.extend(dfs(c, path_so_far, node))

            return result

        psf = []
        return dfs(self, psf, None)






In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

def preprocess(tree):
    paths = Node(tree).downward_paths()
    flat = flatten(paths)
    return ' '.join(flat)


In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [5]:
data = pd.read_pickle('etl/dataset10.pkl')

In [6]:
target = data.hash
trees = data.tree

In [7]:
features = []
for t in trees:
    feats = preprocess(t)
    
    features.append(feats)

In [8]:
X = vectorizer.fit_transform(features)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.33)

In [10]:
rf = RandomForestClassifier(n_estimators=50).fit(X_train,y_train)

In [11]:
rf.score(X_test, y_test)

0.18421052631578946